In [1]:
!git clone 'https://github.com/aakashvardhan/s7-in-depth-practice.git'

Cloning into 's7-in-depth-practice'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 72 (delta 29), reused 63 (delta 20), pack-reused 0
Receiving objects: 100% (72/72), 173.04 KiB | 24.72 MiB/s, done.
Resolving deltas: 100% (29/29), done.


In [2]:
import sys
sys.path.append('/content/s7-in-depth-practice')
sys.path.append('/content/s7-in-depth-practice/models')

In [3]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from model_utils import model_summary, test_model_sanity, sgd_optimizer
from models.model import model2

from utils import train, test, get_device, plt_fig

In [4]:
random_rotations = 7
img_size = (28,28)

# Train Phase transformations
train_transforms = transforms.Compose([
                                       transforms.RandomResizedCrop(img_size, (0.80, 1.00)),
                                       transforms.RandomRotation(random_rotations),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values.
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])

In [5]:
train_data = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test_data = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

100%|██████████| 9912422/9912422 [00:00<00:00, 266505844.40it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 107010330.23it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 131819144.13it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 17558091.03it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [6]:
# CUDA?
cuda = get_device()

# For reproducibility
torch.manual_seed(1)

if torch.cuda.is_available():
    torch.cuda.manual_seed(1)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train_data, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test_data, **dataloader_args)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [7]:
model2 = model2().to(cuda)

In [8]:
model_summary(model2)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
         ConvBlock-5            [-1, 8, 26, 26]               0
            Conv2d-6           [-1, 16, 24, 24]           1,152
              ReLU-7           [-1, 16, 24, 24]               0
       BatchNorm2d-8           [-1, 16, 24, 24]              32
           Dropout-9           [-1, 16, 24, 24]               0
        ConvBlock-10           [-1, 16, 24, 24]               0
           Conv2d-11           [-1, 16, 22, 22]           2,304
             ReLU-12           [-1, 16, 22, 22]               0
      BatchNorm2d-13           [-1, 16, 22, 22]              32
          Dropout-14           [-1, 16,

In [9]:
from torch.optim.lr_scheduler import StepLR


optimizer = sgd_optimizer(model2, 0.05)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)
EPOCHS = 15
lr = []
for epoch in range(1,EPOCHS+1):
    print("EPOCH:", epoch)
    train(model2, cuda, train_loader, optimizer, epoch)
    test(model2, cuda, test_loader)
    scheduler.step()
    lr.append(optimizer.param_groups[0]['lr'])

EPOCH: 1


Loss=0.08016625791788101 Batch_id=468 Accuracy=90.60: 100%|██████████| 469/469 [00:37<00:00, 12.36it/s]



Test set: Average loss: 0.0666, Accuracy: 9795/10000 (97.95%)

EPOCH: 2


Loss=0.14507955312728882 Batch_id=468 Accuracy=97.07: 100%|██████████| 469/469 [00:29<00:00, 15.98it/s]



Test set: Average loss: 0.0406, Accuracy: 9877/10000 (98.77%)

EPOCH: 3


...


Test set: Average loss: 0.0200, Accuracy: 9932/10000 (99.32%)

EPOCH: 14


Loss=0.010079321451485157 Batch_id=468 Accuracy=98.81: 100%|██████████| 469/469 [00:30<00:00, 15.43it/s]



Test set: Average loss: 0.0199, Accuracy: 9929/10000 (99.29%)

EPOCH: 15


Loss=0.0077639389783144 Batch_id=468 Accuracy=98.81: 100%|██████████| 469/469 [00:31<00:00, 14.99it/s]



Test set: Average loss: 0.0196, Accuracy: 9930/10000 (99.30%)



In [10]:
# Save model
torch.save(model2.state_dict(), 'best_model.pth')